In [1]:
%run helper_functions.ipynb

In [2]:
import pandas as pd

In [3]:
%%time

transactions = pd.read_parquet('data/transactions_train.parquet')
customers = pd.read_parquet('data/customers.parquet')
articles = pd.read_parquet('data/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 1.57 s, sys: 634 ms, total: 2.21 s
Wall time: 734 ms


In [4]:
test_week = transactions.week.max()
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [5]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 11.6 s, sys: 652 ms, total: 12.3 s
Wall time: 11.5 s


In [6]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 403 ms, sys: 24 ms, total: 427 ms
Wall time: 426 ms


In [7]:
candidates_last_purchase = transactions.copy()

In [8]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 5.6 s, sys: 30.9 ms, total: 5.63 s
Wall time: 5.63 s


### Bestsellers candidates

In [9]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [10]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [11]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [12]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [13]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [14]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [15]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [16]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [17]:
transactions['purchased'] = 1

In [18]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

data.purchased.mean()

0.14606530579445656

In [19]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

### Add bestseller information

In [20]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [21]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [22]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [23]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [24]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [25]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [26]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [27]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

CPU times: user 91.8 ms, sys: 48.2 ms, total: 140 ms
Wall time: 140 ms


In [28]:
test_baskets = test.groupby(['week', 'customer_id'])['article_id'].count().values

In [29]:
test_X = test[columns_to_use]
test_y = test['purchased']

# Model training

In [30]:
from lightgbm.sklearn import LGBMRanker

In [31]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=3,
    importance_type='gain',
    verbose=10,
)

In [32]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
    eval_set=[(test_X, test_y)],
    eval_group=[test_baskets]
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.849275
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.152439
[LightGBM] [Debug] init for col-wise cost 0.109064 seconds, init for row-wise cost 0.206169 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.188541 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1144
[LightGBM] [Info] Number of data points in the train set: 10205052, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[1]	valid_0's ndcg@1: 0.969401	valid_0's ndcg@2: 0.973003	valid_0's ndcg@3: 0.976278	valid_0's ndcg@4: 0.978733	valid_0's ndcg@5: 0.980428
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[2]	valid_0's ndcg@1: 0.962583	valid_0's ndcg@2: 0.966608	valid_0's 

In [33]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9849463754791337
article_id 0.007059839724299058
garment_group_no 0.0020878366198953096
product_type_no 0.0020186742698389526
department_no 0.001396019268052196
colour_group_code 0.0006179027844180877
club_member_status 0.0005309552726249293
postal_code 0.0005162133320389338
graphical_appearance_no 0.00028952994909009754
Active 0.0002736160490724156
age 0.000118185409108431
index_group_no 7.017324790632495e-05
index_code 6.74223935270559e-05
perceived_colour_value_id 7.256200994568401e-06
fashion_news_frequency 0.0
FN 0.0
section_no 0.0
perceived_colour_master_id 0.0
